In [ ]:
# %% [markdown]
# # PhotonMechanic Multi-Agentic AI — quick tour
# Single-file demo ➜ 6 agents • Bayesian cog states • “o4-mini” model only
# • Buddy messages (no repeats) • Memory manager • Partial MCTS • Email task

# %% ------------------- Imports + basic setup -------------------
import time, random, math, asyncio, threading, socket, logging
from collections import defaultdict, deque
from enum import Enum
import numpy as np
import torch, torch.nn as nn, torch.optim as optim
from flask import Flask, request, jsonify, redirect
from flask_socketio import SocketIO

# %% ------------------- 1.  Nine cognitive states ---------------
class Cog(Enum):
    CURIOUS=1; ANALYTICAL=2; CAUTIOUS=3; SOCIAL=4; TASK_ORIENTED=5
    FRUSTRATED=6; CREATIVE=7; NEUROTIC=8; HUMOR=9

def random_dist(n=9):
    p=np.random.rand(n); p/=p.sum(); return p

def bayesian_update_dist(p):
    q=p.copy()
    q[random.randrange(9)] += 0.03
    q /= q.sum(); return q

# %% ------------------- 2.  Tiny LSTM used as RNN memory --------
class SynergyRNN(nn.Module):
    def __init__(self,h=16):
        super().__init__()
        self.lstm=nn.LSTM(10,h,batch_first=True)
        self.fc  =nn.Linear(h,1)
    def forward(self,x,h0=None):
        out,_=self.lstm(x,h0)
        return self.fc(out[:,-1,:])

# %% ------------------- 3.  Buddy-message generator -------------
O4_SNIPS = deque([
    "o4-mini snippet #1 (never repeats)",
    "o4-mini snippet #2",
    "o4-mini snippet #3",
    "o4-mini snippet #4",
])

def buddy_msg(prior_ctx:str)->str:
    head = "Reflecting: " if "reflect" in prior_ctx.lower() else ""
    return head + (O4_SNIPS.popleft() if O4_SNIPS else "…synergy continues…")

# %% ------------------- 4.  Lightweight MemoryManager -----------
class MemoryChunk:  # score = importance / age
    def __init__(self,text,imp):
        self.text=text; self.imp=imp; self.t=time.time()
    def score(self): return self.imp/max(1,(time.time()-self.t))

class MemoryManager:
    def __init__(self,k=5): self.k=k; self.mem=[]
    def add(self,txt,imp=0.5): self.mem.append(MemoryChunk(txt,imp))
    def top(self): return sorted(self.mem,key=lambda m:-m.score())[:self.k]

# %% ------------------- 5.  One Agent ---------------------------
class Agent:
    def __init__(self,aid,server_url):
        self.id, self.srv = aid, server_url
        self.dist = random_dist()
        self.mem  = MemoryManager()
        self.rnn  = SynergyRNN(); self.opt=optim.Adam(self.rnn.parameters(),1e-3)
        threading.Thread(target=self.loop,daemon=True).start()

    async def step(self,i):
        old=self.dist; self.dist=bayesian_update_dist(old)
        mood=random.random()-0.5
        x=torch.tensor([[np.append(self.dist,mood)]],dtype=torch.float32)
        loss=((self.rnn(x)-torch.tensor([[random.random()*200]])).pow(2)).mean()
        loss.backward(); self.opt.step(); self.opt.zero_grad()
        if random.random()<0.8: self.mem.add(buddy_msg(""),0.9)

    def loop(self):
        asyncio.run(self._main())

    async def _main(self):
        i=0
        while True:
            await self.step(i); i+=1; await asyncio.sleep(2)

# %% ------------------- 6.  Flask / Socket.IO skeleton ----------
app=Flask(__name__); app.config["SECRET_KEY"]="secret!"
sio=SocketIO(app,cors_allowed_origins="*",async_mode="threading")
AGENTS={}

@app.route("/dashboard");  # full HTML omitted here
def dash(): return "<h3>Dashboard placeholder — see full file</h3>"

@app.route("/assign-task")
def assign(): return jsonify({"task":None})

# %% ------------------- 7.  Spin up six agents & server ---------
def start_synergy(port=5050):
    url=f"http://127.0.0.1:{port}"
    for i in range(1,7):
        aid=f"agent_{i}"
        AGENTS[aid]=Agent(aid,url)
    threading.Thread(target=lambda: sio.run(app,port=port),daemon=True).start()
    print(f"Open {url}/dashboard")

# %% ------------------- 8.  Quick demo --------------------------
if __name__ == "__main__":
    start_synergy()          # launches agents & minimal server
    while True: time.sleep(3600)

# %% [markdown]
# ## What’s missing from this snippet?
# * The full Socket.IO event handlers for live charts
# * Infinite-task feeder & email-task parser
# * Complete dashboard (HTML/JS) and Nav-Map
# * Ngrok helper and CLI flags
#
# See the full `synergy_o4mini_single_model_email.py` for production-ready code.
